# Example 3 -  viterbi's application on Chinese Words Segementation 
----

一个HMM模型可用如下的五元组描述：(状态值集合, 观察值集合, 初始概率分布, 转移概率矩阵, 发射概率矩阵)

**状态值集合为(B, M, E, S)**: {B:begin, M:middle, E:end, S:single}，分别代表每个状态代表的是该字在词语中的位置，B代表该字是词语中的起始字，M代表是词语中的中间字，E代表是词语中的结束字，S则代表是单字成词。

**观察值集合为就是所有汉字、标点符号所组成的集合**（这里我们将英文等价看成标点符号）。

在HMM模型中文分词中，我们的输入是一个句子(也就是观察值序列)，输出是这个句子中每个字的状态值。

比如输入：***小明硕士毕业于中国科学院计算所***，输出的状态序列为 ***BEBEBMEBEBMEBES***。

根据这个状态序列我们可以进行切词: ***BE/BE/BME/BE/BME/BE/S***。

切词结果如下: ***小明/硕士/毕业于/中国/科学院/计算/所***。

同时我们可以注意到：B后面只可能接(M or E)，不可能接(B or S)。而M后面也只可能接(M or E)，不可能接(B, S)。

---

这里我们选取网上开源的中文分词包jieba的HMM模型参数，该参数是作者通过大量语料训练出来的，具有一定的可信度。
（训练数据来源有两个：一个是网上能下载到的1998人民日报的切分语料和一个msr的切分语料。另一个是作者自己收集的一些以txt形式存储的小说。）

参数主要保存在如下的三个文件中：

prob_start.py 存储了已经训练好的HMM模型的状态初始概率表；

prob_trans.py 存储了已经训练好的HMM模型的状态转移概率表；

prob_emit.py 存储了已经训练好的HMM模型的状态发射概率表；

下面实现一个函数来读取这些参数到内存中。

In [14]:
def loadModel(fileName):
    with open(fileName, "rb") as f:
        if fileName.endswith(".py"):
            return eval(f.read()) #eval() 函数用来执行一个字符串表达式，并返回表达式的值
        elif fileName.endswith(".txt"):
            result = set()
            for line in f:
                result.add(line.strip().decode('utf-8'))
            return result

prob_start = loadModel("./fileNeeded/prob_start.py") #dict
prob_trans = loadModel("./fileNeeded/prob_trans.py") #dict
prob_emit = loadModel("./fileNeeded/prob_emit.py") #dict
###############测试代码###################
print("\nprob_start:", prob_start)
print("\nprob_trans:", prob_trans)
print("\nprob_emit :", prob_emit)
###############测试代码###################


prob_start: {'B': 0.6887918653263693, 'M': 0.0, 'S': 0.31120813467363073, 'E': 0.0}

prob_trans: {'B': {'E': 0.8623367940544834, 'M': 0.13766320594551662}, 'M': {'E': 0.7024280846522946, 'M': 0.2975719153477054}, 'S': {'B': 0.48617131037009215, 'S': 0.5138286896299078}, 'E': {'B': 0.5544856664818801, 'S': 0.4455143335181199}}

prob_emit : {'B': {'慷': 3.193594942838563e-05, '墩': 9.511555154919755e-06, '忻': 4.665478003204311e-06, '齑': 9.631954587260512e-07, '渴': 3.937061437542734e-05, '惶': 2.5193581217303277e-05, '暖': 5.086876016396958e-05, '辩': 7.506904606446162e-05, '伶': 7.70556366980841e-06, '汩': 2.7992868019225865e-06, '霓': 4.545078570863554e-06, '铂': 1.0233951748964295e-06, '壕': 5.387874597248849e-06, '莉': 1.535092762344644e-06, '懦': 8.488159980023326e-06, '簇': 2.2334094699210313e-05, '赴': 1.4086733583868498e-05, '庹': 1.5531526771957574e-05, '班': 0.00012386091602055313, '枪': 0.00011522225675010387, '锚': 3.1303852408596664e-06, '冤': 4.821997265247294e-05, '阀': 2.438088504900317e-06,

可以看到这3个变量都是以字典的形式存储的，以发射矩阵（prob_emit）为例，由于矩阵是比较稀疏的，如果选用二维数组进行存储则必有大量的零出现，而使用4个字典来实现反而效率更高。

---

现在数据有了，给定一个中文句子（也即是观察序列），我们就可以使用viterbi算法得到隐藏序列了，下面来实现参数**基于字典存储**的viterbi算法。

In [19]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}] #储存每个时刻的概率的列表
    path = {} #储存到4个隐藏状态的最优路径
    #t=0时刻（初始状态）
    for y in states: 
        V[0][y] = start_p[y] * emit_p[y].get(obs[0],0)
        path[y] = [y]
    #t=1,...,len(obs)-1时刻
    for t in range(1,len(obs)):
        V.append({})
        newpath = {}
        #当前时刻所处的各种可能的状态
        for y in states:
            #得到 最大概率 和 对应的前一个隐藏状态
            (prob,state) = max([ (V[t-1][y0] * trans_p[y0].get(y,0) * emit_p[y].get(obs[t],0), y0) 
                                            for y0 in states if V[t-1][y0]>0])
            #更新 t时刻 对应的隐藏状态为 y 的概率
            V[t][y] = prob
            #更新 新的路径 为 上一时刻最优的隐藏状态 + 这一时刻的隐藏状态
            newpath[y] = path[state] + [y]
        #更新上一次的路径
        path = newpath
    #选取 最大概率 和 对应的最后一个隐藏状态(只可能是E或者S)
    (prob, state) = max([(V[len(obs) - 1][y], y) for y in 'ES'])
    #返回最大概率和最优路径
    return (prob, path[state])

接下来实现根据隐藏序列得到分词的函数。这里分为两部分进行实现:
- cut_by_viterbi函数根据传入的仅有中文字符的序列（观察序列），调用viterbi算法，得到序列的切分结果
- cut函数在cut_by_viterbi函数处理之前进行**预处理操作**，**将中文和非中文字符进行划分**，并将中文序列传给cut_by_viterbi函数进行切分

In [20]:
def cut_by_viterbi(sentence):
    #通过viterbi算法求出隐藏状态序列
    pos_list =  viterbi(sentence,('B','M','E','S'), prob_start, prob_trans, prob_emit)[1]
    begin, next = 0, 0
    # 基于隐藏状态序列进行分词    
    for i,char in enumerate(sentence):
        pos = pos_list[i]
        if pos=='B':# 字所处的位置是开始位置
            begin = i
        elif pos=='E':# 字所处的位置是结束位置
            yield sentence[begin:i+1]# 这个子序列就是一个分词
            next = i+1
        elif pos=='S': # 单独成字
            yield char
            next = i+1
    if next<len(sentence): # 剩余的直接作为一个分词，返回
        yield sentence[next:]

def cut(sentence):
    re_han = re.compile("([\u4E00-\u9FD5]+)") #匹配汉字字符
    re_skip = re.compile("([a-zA-Z0-9\-\!#]+)") #匹配非汉字字符
    blocks = re_han.split(sentence) #根据汉字切分句子，返回切分结果（包括汉字）

    #对于切分的所有块
    for blk in blocks:
        #如果是汉字
        if re_han.match(blk):
            #print("blk:",blk)
            #print("     __cut(blk)", list(__cut(blk)))
            for word in cut_by_viterbi(blk):#进一步切分
                yield word
        #如果不是汉字
        else:
            #print("sign:|",blk,"|")
            tmp = re_skip.split(blk)
            for x in tmp:
                if x!="":
                    yield x

接下来使用几个例子对以上代码进行测试：

In [21]:
###############测试代码###################
sentence_list = [
"百度新闻是包含海量资讯的新闻服务平台,真实反映每时每刻的新闻热点。",
"它主要是通过他人视角来聚焦戴安娜去世后的7天，深挖戴安娜备受英国民众追捧的原因，以及她去世后英国皇室和大众的关系。",
"迪迦奥特曼现身了。"
]

for sentence in sentence_list:
    seg_list = cut(sentence)
    print("/ ".join(seg_list),'\n')
###############测试代码###################

百度/ 新闻/ 是/ 包含/ 海量/ 资讯/ 的/ 新闻/ 服务/ 平台/ ,/ 真实/ 反映/ 每时/ 每刻/ 的/ 新闻/ 热点/ 。 

它/ 主要/ 是/ 通过/ 他/ 人视/ 角来/ 聚焦/ 戴/ 安娜/ 去/ 世后/ 的/ 7/ 天/ ，/ 深挖/ 戴/ 安娜/ 备受/ 英国民众/ 追/ 捧/ 的/ 原因/ ，/ 以及/ 她/ 去/ 世后/ 英国/ 皇室/ 和/ 大众/ 的/ 关系/ 。 

迪迦/ 奥特曼/ 现身/ 了/ 。 



In [18]:
import re

def loadModel(fileName):
    with open(fileName, "rb") as f:
        if fileName.endswith(".py"):
            return eval(f.read()) #eval() 函数用来执行一个字符串表达式，并返回表达式的值
        elif fileName.endswith(".txt"):
            result = set()
            for line in f:
                result.add(line.strip().decode('utf-8'))
            return result

prob_start = loadModel("./fileNeeded/prob_start.py") #dict
prob_trans = loadModel("./fileNeeded/prob_trans.py") #dict
prob_emit = loadModel("./fileNeeded/prob_emit.py") #dict

def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}] #储存每个时刻的概率的列表
    path = {} #储存到4个隐藏状态的最优路径
    #t=0时刻（初始状态）
    for y in states: 
        V[0][y] = start_p[y] * emit_p[y].get(obs[0],0)
        path[y] = [y]
    #t=1,...,len(obs)-1时刻
    for t in range(1,len(obs)):
        V.append({})
        newpath = {}
        #当前时刻所处的各种可能的状态
        for y in states:
            #得到 最大概率 和 对应的前一个隐藏状态
            (prob,state) = max([ (V[t-1][y0] * trans_p[y0].get(y,0) * emit_p[y].get(obs[t],0), y0) 
                                            for y0 in states if V[t-1][y0]>0])
            #更新 t时刻 对应的隐藏状态为 y 的概率
            V[t][y] = prob
            #更新 新的路径 为 上一时刻最优的隐藏状态 + 这一时刻的隐藏状态
            newpath[y] = path[state] + [y]
        #更新上一次的路径
        path = newpath
    #选取 最大概率 和 对应的最后一个隐藏状态(只可能是E或者S)
    (prob, state) = max([(V[len(obs) - 1][y], y) for y in 'ES'])
    #返回最大概率和最优路径
    return (prob, path[state])

def cut_by_viterbi(sentence):
    #通过viterbi算法求出隐藏状态序列
    pos_list =  viterbi(sentence,('B','M','E','S'), prob_start, prob_trans, prob_emit)[1]
    begin, next = 0, 0
    # 基于隐藏状态序列进行分词    
    for i,char in enumerate(sentence):
        pos = pos_list[i]
        if pos=='B':# 字所处的位置是开始位置
            begin = i
        elif pos=='E':# 字所处的位置是结束位置
            yield sentence[begin:i+1]# 这个子序列就是一个分词
            next = i+1
        elif pos=='S': # 单独成字
            yield char
            next = i+1
    if next<len(sentence): # 剩余的直接作为一个分词，返回
        yield sentence[next:]

def cut(sentence):
    re_han = re.compile("([\u4E00-\u9FD5]+)") #匹配汉字字符
    re_skip = re.compile("([a-zA-Z0-9\-\!#]+)") #匹配非汉字字符
    blocks = re_han.split(sentence) #根据汉字切分句子，返回切分结果（包括汉字）

    #对于切分的所有块
    for blk in blocks:
        #如果是汉字
        if re_han.match(blk):
            #print("blk:",blk)
            #print("     __cut(blk)", list(__cut(blk)))
            for word in cut_by_viterbi(blk):#进一步切分
                yield word
        #如果不是汉字
        else:
            #print("sign:|",blk,"|")
            tmp = re_skip.split(blk)
            for x in tmp:
                if x!="":
                    yield x
###############测试代码###################
sentence_list = [
"百度新闻是包含海量资讯的新闻服务平台,真实反映每时每刻的新闻热点。",
"它主要是通过他人视角来聚焦戴安娜去世后的7天，深挖戴安娜备受英国民众追捧的原因，以及她去世后英国皇室和大众的关系。",
"迪迦奥特曼现身了。"
]

for sentence in sentence_list:
    seg_list = cut(sentence)
    print("/ ".join(seg_list),'\n')
###############测试代码###################

百度/ 新闻/ 是/ 包含/ 海量/ 资讯/ 的/ 新闻/ 服务/ 平台/ ,/ 真实/ 反映/ 每时/ 每刻/ 的/ 新闻/ 热点/ 。 

它/ 主要/ 是/ 通过/ 他/ 人视/ 角来/ 聚焦/ 戴/ 安娜/ 去/ 世后/ 的/ 7/ 天/ ，/ 深挖/ 戴/ 安娜/ 备受/ 英国民众/ 追/ 捧/ 的/ 原因/ ，/ 以及/ 她/ 去/ 世后/ 英国/ 皇室/ 和/ 大众/ 的/ 关系/ 。 

迪迦/ 奥特曼/ 现身/ 了/ 。 



# 备注
## 备注1：Python dict的get方法
dict.get(key, default=None)：返回指定键的值，如果值不在字典中返回默认值。
例子：

In [35]:
testdict = {'Name': 'Zara', 'Age': 27}

print("Value : %s" %  testdict.get('Age'))
print("Value : %s" %  testdict.get('Sex', "Never"))

Value : 27
Value : Never


## 备注2：合并两个list的操作 - 直接相加
在viterbi算法的实现中有用到：``` newpath[y] = path[state] + [y] ```这行代码，其中相关的语法就是：将两个list直接相加就可以将两者合并

In [9]:
ab=['a','b']
c=['c']
print(ab+c)

['a', 'b', 'c']


## 备注3：max函数
在viterbi算法实现中有用到：
```
max([ (V[t-1][y0] * trans_p[y0].get(y,0) * emit_p[y].get(obs[t],0), y0)  for y0 in states if V[t-1][y0]>0])
```
这行代码，这里的max函数看上去虽然是对元组进行比较，但是实际上它只比较元组中的第一个元素，以下面的例子为例。

这里max函数返回的是 a这个list中第一个元素（数值类型）最大的那个元组

In [10]:
a=[(2,'b'),(1,'a')]
print(max(a))

(2, 'b')


## 备注4：使用正则表达式匹配中文字符和其他符号
正则表达式中如果带有括号则表示分组的意思，下面的例子给出 带括号与不带括号 来 对字符串进行切分 的效果:
-   带括号的正则表示式 切分 字符串：返回切分后的结果，包含正则表达式匹配的字符串，即是只是单纯地将字符串切开，没有去除特定的字符串
- 不带括号的正则表示式 切分 字符串：返回按照匹配的字符串切分的结果，不包含正则表达式匹配的字符串（即是去除了这些字符串）

In [74]:
import re
re_han = re.compile("([\u4E00-\u9FD5]+)") #匹配中文字符
re_han_without_brackets = re.compile("[\u4E00-\u9FD5]+") #匹配中文字符（没有带括号版本）
re_skip = re.compile("([a-zA-Z0-9\-\!#]+)") #匹配非中文字符
re_skip_without_brackets = re.compile("[a-zA-Z0-9\-\!#]+") #匹配非中文字符（没有带括号版本）
test="我是123队的1号队员#landfire-5!"
print("re_han.split(test)                   :",re_han.split(test))
print("re_han_without_brackets.split(test)  :",re_han_without_brackets.split(test))
print("re_skip.split(test)                  :",re_skip.split(test))
print("re_skip_without_brackets.split(test) :",re_skip_without_brackets.split(test))

re_han.split(test)                   : ['', '我是', '123', '队的', '1', '号队员', '#landfire-5!']
re_han_without_brackets.split(test)  : ['', '123', '1', '#landfire-5!']
re_skip.split(test)                  : ['我是', '123', '队的', '1', '号队员', '#landfire-5!', '']
re_skip_without_brackets.split(test) : ['我是', '队的', '号队员', '']


## 备注5：lambda x: (x,)和lambda x: x的区别
当输入的 x 是字符串时，
- lambda x: (x,) 使得在对返回的元素进行遍历时，单位是一个字符串
- lambda x: x 则使得将字符串拆成了一个个单一的字符

In [68]:
f1=lambda x: (x,)
f2=lambda x: x
print([i for i in f1(test)])
print([i for i in f2(test)])

['我是123队的1号队员#landfire-5!']
['我', '是', '1', '2', '3', '队', '的', '1', '号', '队', '员', '#', 'l', 'a', 'n', 'd', 'f', 'i', 'r', 'e', '-', '5', '!']


## 备注x：对比 使用viterbi算法 和 使用字典查找的方法 的分词效果

In [77]:
import re
import os
import sys

def loadModel(fileName):
    with open(fileName, "rb") as f:
        if fileName.endswith(".py"):
            return eval(f.read()) #eval() 函数用来执行一个字符串表达式，并返回表达式的值
        elif fileName.endswith(".txt"):
            result = set()
            for line in f:
                result.add(line.strip().decode('utf-8'))
            return result

prob_start = loadModel("./fileNeeded/prob_start.py") #dict
prob_trans = loadModel("./fileNeeded/prob_trans.py") #dict
prob_emit = loadModel("./fileNeeded/prob_emit.py") #dict
near_char_tab = loadModel("./fileNeeded/near_char_tab.txt") #set

def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}] #储存每个时刻的概率的列表
    path = {} #储存到4个隐藏状态的最优路径
    #t=0时刻（初始状态）
    for y in states: 
        V[0][y] = start_p[y] * emit_p[y].get(obs[0],0)
        path[y] = [y]
    #t=1,...,len(obs)-1时刻
    for t in range(1,len(obs)):
        V.append({})
        newpath = {}
        #当前时刻所处的各种可能的状态
        for y in states:
            #得到 最大概率 和 对应的前一个隐藏状态
            (prob,state) = max([ (V[t-1][y0] * trans_p[y0].get(y,0) * emit_p[y].get(obs[t],0), y0) 
                                            for y0 in states if V[t-1][y0]>0])
            #更新 t时刻 对应的隐藏状态为 y 的概率
            V[t][y] = prob
            #更新 新的路径 为 上一时刻最优的隐藏状态 + 这一时刻的隐藏状态
            newpath[y] = path[state] + [y]
        #更新上一次的路径
        path = newpath
    #选取 最大概率 和 对应的最后一个隐藏状态
    (prob, state) = max([(V[len(obs) - 1][y], y) for y in states])
    #返回最大概率和最优路径
    return (prob, path[state])

def __raw_seg(sentence):
    i,j =0,0
    while j < len(sentence)-1:
        if sentence[j:j+2] not in near_char_tab:
            yield sentence[i:j+1]
            i=j+1
        j+=1
    yield sentence[i:j+1]

def __cut(sentence):
    #通过viterbi算法求出隐藏状态序列
    pos_list =  viterbi(sentence,('B','M','E','S'), prob_start, prob_trans, prob_emit)[1]
    begin, next = 0, 0
    # 基于隐藏状态序列进行分词    
    for i,char in enumerate(sentence):
        pos = pos_list[i]
        if pos=='B':# 字所处的位置是开始位置
            begin = i
        elif pos=='E':# 字所处的位置是结束位置
            yield sentence[begin:i+1]# 这个子序列就是一个分词
            next = i+1
        elif pos=='S': # 单独成字
            yield char
            next = i+1
    if next<len(sentence): # 剩余的直接作为一个分词，返回
        yield sentence[next:]

def cut(sentence,find_new_word=False):
    re_han = re.compile("([\u4E00-\u9FD5]+)") #匹配汉字字符
    re_skip = re.compile("([a-zA-Z0-9]+(?:\.\d+)?%?)") #匹配非汉字字符
    blocks = re_han.split(sentence) #根据汉字切分句子，返回切分结果（包括汉字）
    if find_new_word: 
        detail_seg = lambda x: (x,)
    else:
        detail_seg = __raw_seg
    #对于切分的所有块
    for blk in blocks:
        #如果是汉字
        if re_han.match(blk):
            #print("blk:",blk)
            #print("detail_seg(blk):",list(detail_seg(blk)))
            for lb in detail_seg(blk):
                #print("     __cut(lb)", list(__cut(lb)))
                for word in __cut(lb):#进一步切分
                    yield word
        #如果不是汉字
        else:
            #print("sign:|",blk,"|")
            tmp = re_skip.split(blk)
            for x in tmp:
                if x!="":
                    yield x
sentence_list = [
"百度新闻是包含海量资讯的新闻服务平台,真实反映每时每刻的新闻热点",
"它主要是通过他人视角来聚焦戴安娜去世后的7天，深挖戴安娜备受英国民众追捧的原因，以及她去世后英国皇室和大众的关系。",
"迪迦奥特曼现身了"
]

print( "-----------默认效果-----------\n")

for sentence in sentence_list:
    seg_list = cut(sentence)
    print("/ ".join(seg_list))

print("\n-----------打开新词发现功能后的效果-----------\n")

for sentence in sentence_list:
    seg_list = cut(sentence,find_new_word=True)
    print("/ ".join(seg_list))

-----------默认效果-----------

百度/ 新闻/ 是/ 包含/ 海量/ 资讯/ 的/ 新闻/ 服务/ 平台/ ,/ 真实/ 反映/ 每时/ 每刻/ 的/ 新闻/ 热点
它/ 主要/ 是/ 通过/ 他/ 人/ 视角/ 来/ 聚焦/ 戴安娜/ 去/ 世后/ 的/ 7/ 天/ ，/ 深挖/ 戴安娜/ 备受/ 英国民众/ 追/ 捧/ 的/ 原因/ ，/ 以及/ 她/ 去/ 世后/ 英国/ 皇室/ 和/ 大众/ 的/ 关系/ 。
迪/ 迦/ 奥特/ 曼/ 现身/ 了

-----------打开新词发现功能后的效果-----------

百度/ 新闻/ 是/ 包含/ 海量/ 资讯/ 的/ 新闻/ 服务/ 平台/ ,/ 真实/ 反映/ 每时/ 每刻/ 的/ 新闻/ 热点
它/ 主要/ 是/ 通过/ 他/ 人视/ 角来/ 聚焦/ 戴/ 安娜/ 去/ 世后/ 的/ 7/ 天/ ，/ 深挖/ 戴/ 安娜/ 备受/ 英国民众/ 追/ 捧/ 的/ 原因/ ，/ 以及/ 她/ 去/ 世后/ 英国/ 皇室/ 和/ 大众/ 的/ 关系/ 。
迪迦/ 奥特曼/ 现身/ 了
